In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import tensorflow as tf
from glob import glob
repo_path = ".."
import sys
sys.path.append(f"{repo_path}/code/")

from template_model import TemplateModel

font = {'size'   : 14}

matplotlib.rc('font', **font)
figure = {'figsize'   : (12,8),
          'max_open_warning': False}
matplotlib.rc('figure', **figure)

In [ ]:
inf_re = "../data/models/b_2/ne_200_lr_1e-06_bs_1000_t_0.1/init_[0-9]"
clf_re = "../data/models/cross_entropy/ne_200_lr_0.001_bs_32/init_[0-9]"

pars = ["s_exp","r_dist","b_rate"]

In [ ]:

sess = tf.Session()

r_dist_scan = np.linspace(1.7,2.3,13,endpoint=True)

tm = TemplateModel ()

inf_margs = {}
for inf_path in glob(inf_re):
  inf_margs[inf_path] = []
  for r_dist in r_dist_scan:
    tm.templates_from_json(f"{inf_path}/templates.json",
                           r_dist=np.round([r_dist, r_dist+0.2,  r_dist-0.2],
                                           decimals=2))
    fisher_matrix = tm.asimov_hess(sess=sess)
    inf_margs[inf_path].append(fisher_matrix.marginals(pars)["s_exp"])
    

clf_margs = {}
for clf_path in glob(clf_re):
  clf_margs[clf_path] = []
  for r_dist in r_dist_scan:
    tm.templates_from_json(f"{clf_path}/templates.json",
                           r_dist=np.round([r_dist, r_dist+0.2,  r_dist-0.2],
                                           decimals=2))
    fisher_matrix = tm.asimov_hess(sess=sess)
    clf_margs[clf_path].append(fisher_matrix.marginals(pars)["s_exp"])


In [ ]:
fig, ax =  plt.subplots(figsize=(8,6))

for inf_marg in inf_margs.values():
  inf_line = ax.plot(r_dist_scan, inf_marg,"b .",alpha=0.5)
  
for clf_marg in clf_margs.values():
  clf_line =  ax.plot(r_dist_scan, clf_marg,"r .",alpha=0.5)

ax.set_xlabel(r"$r$ value")
ax.set_ylabel(r"expected uncertainty on $s$")

ax.set_ylim([16.,28.])

ax.legend((clf_line[0], inf_line[0]), ("cross-entropy","inference-aware"),
          loc="upper center",frameon=False)
fig.savefig("../paper/gfx/figure5a.pdf",bbox_inches='tight')
fig

In [ ]:

sess = tf.Session()

b_rate_scan = np.linspace(2.5,3.5,11,endpoint=True)


tm = TemplateModel ()

inf_margs = {}
for inf_path in glob(inf_re):
  inf_margs[inf_path] = []
  for b_rate in b_rate_scan:
    tm.templates_from_json(f"{inf_path}/templates.json",
                           b_rate=np.round([b_rate, b_rate+0.5,  b_rate-0.5],
                                           decimals=1))
    fisher_matrix = tm.asimov_hess(sess=sess)
    inf_margs[inf_path].append(fisher_matrix.marginals(pars)["s_exp"])
    

clf_margs = {}
for clf_path in glob(clf_re):
  clf_margs[clf_path] = []
  for b_rate in b_rate_scan:
    tm.templates_from_json(f"{clf_path}/templates.json",
                           b_rate=np.round([b_rate, b_rate+0.5,  b_rate-0.5],
                                           decimals=1))
    fisher_matrix = tm.asimov_hess(sess=sess)
    clf_margs[clf_path].append(fisher_matrix.marginals(pars)["s_exp"])

In [ ]:
fig, ax =  plt.subplots(figsize=(8,6))

for inf_marg in inf_margs.values():
  inf_line = ax.plot(b_rate_scan, inf_marg,"b .",alpha=0.5)
  
for clf_marg in clf_margs.values():
  clf_line = ax.plot(b_rate_scan, clf_marg,"r .",alpha=0.5)

ax.set_xlabel(r"$\lambda$ value")
ax.set_ylabel(r"expected uncertainty on $s$")


ax.set_ylim([16.,28.])
ax.legend((clf_line[0], inf_line[0]), ("cross-entropy","inference-aware"),
          loc="upper center",frameon=False)

fig.savefig("../paper/gfx/figure5b.pdf",bbox_inches='tight')
fig